## Using API classes to get data and save to db

# <span style="color:red">clear all output before saving: db output contains passwords! </span>
 
- importing modules needed
- creates a temporary db using .env
- creates the API objects for each vendor/station type
- pulls data from the vendor API


In [ ]:
%load_ext autoreload
%autoreload 2

from ewxpwsdb.db.models import WeatherStation, APIResponse, Reading, StationType
from ewxpwsdb.db.importdata import import_station_file, read_station_table
station_file = '../data/test_stations.tsv'

### optional: create new temp database to work with

In [ ]:

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from sqlmodel import select, delete

db_url = get_db_url()
print(db_url)
engine = get_engine(db_url)
init_db(engine)

# import, using the global 'engine' defined in database module
import_station_file(station_file)

## Create WeatherStation object

In [ ]:
station_type = 'SPECTRUM'


In [ ]:

# conventience function get the first station from the database for a specific type
def get_one_station(station_type, engine = engine):
    with Session(engine) as session:
        statement = select(WeatherStation).where(WeatherStation.station_type == station_type)
        results = session.exec(statement)
        weather_station = results.first()

    return(weather_station)





### create station object from database

In [ ]:
station = get_one_station(station_type)
station

### OR create weather station object without database

In [ ]:
# stations = read_station_table(station_file)
# # add code to find the correct one
# station_data = list(filter(lambda x: (x['station_type']==station_type), stations))[0]
# station = WeatherStation.model_validate(station_data) 
# station.station_type


## Test APIs


In [ ]:
# create API class from station
from ewxpwsdb.weather_apis import API_CLASS_TYPES
wapi = API_CLASS_TYPES[station.station_type](station)
wapi.station_type

In [ ]:
print(wapi.station_type)
print(wapi.weather_station.id)
print(wapi.sampling_interval)
print(wapi.APIConfigClass)

In [ ]:
# check that configuration class is instantiated with same data in database
api_config = wapi.APIConfigClass.model_validate_json_str(wapi.weather_station.api_config)
print(api_config == wapi.api_config)

Get the data from an API request, and save that API_response into the database

In [ ]:
r = wapi.get_readings()


In [ ]:
# check that there is no error
r[0].response_text

compare what the WeatheAPI created with our API Response model

In [ ]:
from ewxpwsdb.db.models import APIResponse

api_response_records = None
api_response_records = api_response_records or wapi.current_api_response_records
print(len(api_response_records))
api_response_record = api_response_records[0]
isinstance(api_response_record, APIResponse)


In [ ]:
wapi.weather_station

In [ ]:
api_response_record.weatherstation_id

save the api responses from the request in the database, which then assigns and ID number(s)

In [ ]:
with Session(engine) as session:
    for arr in api_response_records:
        print(arr.id)
        session.add(arr)
        session.commit()
        print(arr.id)
    

In [ ]:
wapi.current_api_response_records[0].id

transform/harmonize the response data into sensor values.  

In [ ]:
sensor_data =  wapi._transform(api_response_record.response_text)
sensor_data

check that these data can be turned into a Reading object (data + metadata )

In [ ]:
reading = Reading.model_validate_from_station(sensor_data[0], api_response_record)
reading

save the rows of data from the sensor into the database using a Session

In [ ]:
with Session(engine) as session:
    readings = [Reading.model_validate_from_station(data, api_response_record) for data in sensor_data]
    session.bulk_save_objects(readings)    
    session.commit()
    

In [ ]:
station.id

In [ ]:
# summarize readings in the database
station_id = station.id
with Session(engine) as session:
    stmt = select(Reading, WeatherStation).join(WeatherStation).where(WeatherStation.id  == 3)
    results = session.exec(stmt)
    for reading in results:
        print(reading)




In [ ]:
isinstance(readings, list)

## Clean up 

If using databases, remove test databases

In [ ]:
# if sqlite
import re
if re.match('sqlite', get_db_url()):
    from os import remove
    remove('ewxpws.db')

In [46]:
# if postgresl
import re
from sqlmodel import delete, text
drop_stmt = text("""drop database ewxpws""")

if re.match('postgres', get_db_url()):
    with Session(engine) as session:
        session.exec(delete(Reading))
        session.exec(delete(APIResponse))
        session.exec(delete(WeatherStation))
        session.exec(delete(StationType))

        session.commit()

    # con = engine.connect()
    # con.execute(drop_stmt)
    # con.close()


2024-02-01 08:58:48,500 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-01 08:58:48,501 INFO sqlalchemy.engine.Engine DELETE FROM reading
2024-02-01 08:58:48,501 INFO sqlalchemy.engine.Engine [cached since 40.71s ago] {}
2024-02-01 08:58:48,502 INFO sqlalchemy.engine.Engine DELETE FROM apiresponse
2024-02-01 08:58:48,503 INFO sqlalchemy.engine.Engine [generated in 0.00023s] {}
2024-02-01 08:58:48,504 INFO sqlalchemy.engine.Engine DELETE FROM weatherstation
2024-02-01 08:58:48,504 INFO sqlalchemy.engine.Engine [generated in 0.00027s] {}
2024-02-01 08:58:48,505 INFO sqlalchemy.engine.Engine DELETE FROM stationtype
2024-02-01 08:58:48,505 INFO sqlalchemy.engine.Engine [cached since 40.71s ago] {}
2024-02-01 08:58:48,506 INFO sqlalchemy.engine.Engine COMMIT


In [47]:
session.close()
engine.dispose()